In [ ]:
!pip install bokeh
!pip install --upgrade ipywidgets

In [ ]:
from ipywidgets import interact, FloatSlider, interactive_output, BoundedIntText, interactive, GridspecLayout, Dropdown, Layout
import numpy as np

import sys
import argparse

# ML4CVD Imports
from ml4cvd.arguments import parse_args
from sklearn.decomposition import PCA

from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
from ml4cvd.models import make_multimodal_multitask_model, train_model_from_generators
from ml4cvd.tensor_generators import test_train_valid_tensor_generators, big_batch_from_minibatch_generator
from ml4cvd.recipes import train_multimodal_multitask
import matplotlib.pyplot as plt
from itertools import product
%matplotlib inline
output_notebook()

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
def command_to_text(command):
    return command.split(' ')
sys.argv = command_to_text('train --tensors /mnt/disks/ecg-bike-tensors --input_tensors ecg-bike-pretest-short --output_tensors ecg-bike-pretest-short --dense_blocks --conv_layers 32 24 16 --max_pools 2 2 2 --training_steps 75 --validation_steps 15 --test_steps 5 --epochs 100 --patience 1000 --batch_size 64 --output_folder /home/ndiamant/train_runs/ --id vae_ecg_short_for_explore --dense_layers 16 --anneal_rate .01 --anneal_max .0001 --anneal_shift -500 --cache_size 5e8 --variational --decompose')

args = parse_args()

In [ ]:
m, enc, dec = make_multimodal_multitask_model(**args.__dict__)

In [ ]:
args.model = m
args.run_id = args.id
del args.model_file
with test_train_valid_tensor_generators(**args.__dict__) as (generate_train, generate_valid, generate_test):
    train_model_from_generators(generate_train=generate_train, generate_test=generate_test, generate_valid=generate_valid, **args.__dict__)

In [ ]:
x_test_in, x_test_out = big_batch_from_minibatch_generator(args.tensor_maps_in, args.tensor_maps_out, generate_test, minibatches=160, keep_paths=False)
pred = m.predict(x_test_in)
encoded = enc.predict(x_test_in)

In [ ]:
for i in range(6):
    plt.title(f'Latent dim {i} distribution.')
    plt.hist(encoded[:, i], bins=25)
    plt.show()
    plt.plot(x_test_in['input_full_ecg_bike'][i, :, 0], label='reconstructed')
    plt.plot(pred[i, :, 0], c='k', linestyle='--', label='og')
    plt.show()

In [ ]:
p = figure(title="ECG reconstruction", plot_height=400, plot_width=1500, y_range=(-2, 5),
           background_fill_color='#efefef')
x = np.arange(1024)
y = pred[0, :, 0]
r = p.line(x, y, color="#8888cc", line_width=1.5, alpha=0.8, legend_label='Reconstruction from Slider Values')
og_recon = p.line(x, y, color='black', line_width=1, line_dash='dashed', legend_label='Sample Reconstruction')
og = p.line(x, y, color='red', line_width=1, line_dash='dotted', legend_label='Original Sample')
p.legend.click_policy="hide"
ndim = encoded.shape[-1]

class Updater:    
    def __init__(self):
        self.sliders = [
            FloatSlider(min=-3, max=3, step=.01, value=encoded[0, i], description=f'latent {i}') 
            for i in range(ndim)
        ]

        self.sample_slider = BoundedIntText(min=0, max=len(pred)-1)
        self.channel_slider = Dropdown(options=list(range(pred.shape[-1])))
        for slider in self.sliders:
            slider.observe(self._slider_callback, names='value')
        self.prev_sample = 0
        self.__name__ = 'update_latent_values'
        
    def _slider_callback(self, change):
        self(sample=self.sample_slider.value, channel=self.channel_slider.value)
    
    #self = None is gross, but necessary to avoid error in interact
    def __call__(self=None, sample=0, channel=0):
        if self.prev_sample != sample:
            for i, slider in enumerate(self.sliders):
                slider.value = encoded[sample, i]
            self.prev_sample = sample
        z = np.array([[slider.value for slider in self.sliders]])
        recon = dec.predict(z)
        r.data_source.data['y'] = recon[0, :, channel]
        og_recon.data_source.data['y'] = pred[sample, :, channel]
        og.data_source.data['y'] = x_test_in['input_full_ecg_bike'][sample, :, channel]
        push_notebook()

In [ ]:
show(p, notebook_handle=True)
update = Updater()
interact(update, sample=update.sample_slider, channel=update.channel_slider)
grid = GridspecLayout(ndim // 4, 4)
for k, (i, j) in enumerate(product(range(ndim // 4), range(4))):
    grid[i, j] = update.sliders[k]
grid

In [ ]:
pca = PCA()
pca.fit(encoded)
plt.plot(pca.explained_variance_ratio_)

In [ ]:
plt.figure(figsize=(10, 7))
sum_variance = pca.explained_variance_ratio_.cumsum()
plt.plot(sum_variance)
plt.axhline(.9, c='g', linestyle='--', label='90% variance explained')
pct_90 = np.where(sum_variance > .9)[0][0]
plt.axvline(pct_90, c='r', linestyle='--', label=f'# pcs for 90% variance explained: {pct_90}')
plt.legend()
plt.show()